In [473]:
import pandas as pd
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import datetime
devengine = create_engine("sqlite:///C:/Git/tennis_atp/database/bets_sqllite.db")
time_now = datetime.datetime.now()
time_now_formatted = time_now.strftime("%Y-%m-%d")

In [474]:
data=pd.read_sql_query(
    f"Select DISTINCT Winner,Dog,Dog_Record,Winner_Odds,Fav_Record,percent,percent2,Fav_Games,Dog_Games,Fav_Odds,Fav,sex From results_clay1 --where Date > '2023-01-01' and Date not like '{time_now_formatted}'",
    con=devengine,)
data['Fav_Odds']=data['Fav_Odds'].astype(float)
data=data[data['Fav_Odds']>1.19]

In [475]:
data3=data[(data['Fav_Odds']>1.19)&(data['Fav_Odds']<1.35)&(data['percent2']<0.4)&(data['percent']>0.85)&(data['Dog_Games']>4)]
len(data3[data3['Winner']==data3['Fav']])/len(data3)

0.84

In [476]:
len(data3)

25

In [477]:
data3[data3['Winner']!=data3['Fav']]

,Winner,Dog,Dog_Record,Winner_Odds,Fav_Record,percent,percent2,Fav_Games,Dog_Games,Fav_Odds,Fav,Sex
112,Tomas Farjat,Tomas Farjat,0%,3.29,90%,0.9,0.00,10,5,1.31,Francisco Comesana,Mens
433,Gerald Melzer,Gerald Melzer,25%,3.34,90%,0.9,0.25,10,16,1.30,Juan Pablo Varillas,Mens
1838,Pedro Sousa,Pedro Sousa,25%,3.11,100%,1.0,0.25,10,8,1.34,Facundo Bagnis,Mens
2717,Renzo Olivo,Renzo Olivo,0%,3.9,90%,0.9,0.00,10,6,1.24,Juan Pablo Varillas,Mens


In [478]:
data1=data[(data['percent2']>0.79)&(data['percent']<0.4)]#&(data['Dog_Games']>4)]#&(data['Sex']=='Womens')]
#data1=data[(data['Fav_Odds']<1.8)&(data['percent2']>0.5)&(data['percent']<0.2)&(data['Dog_Games']>6)]
len(data1[data1['Winner']==data1['Dog']])/len(data1)

0.5333333333333333

In [479]:
print(len(data1))

15
